# 导入

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.dates as mdates
import plotly.graph_objects as go
import webbrowser
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import linregress  # 用于计算趋势
from datetime import datetime, timedelta
from matplotlib.dates import DateFormatter
from plotly.subplots import make_subplots


plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

#####   事件绘图函数（初次绘制）

In [3]:
def plot_time_series_by_periods(
    ts_file_path,          # 时间序列文件路径
    periods_file_path,     # 时间段文件路径
    target_columns,        # 要绘制的列（列表形式）
    output_dir_hourly,            # 输出图片的目录
    output_dir_daily
):
    """   
    参数:
        ts_file_path: 时间序列文件路径
        periods_file_path: 时间段文件路径
        target_columns: 要绘制的列名列表
        output_dir: 输出图片的目录
    """
    
    # 读取数据
    ts_data = pd.read_csv(ts_file_path, parse_dates=['时间'])
    periods_data = pd.read_csv(periods_file_path, parse_dates=['开始时间', '结束时间'])
    
    # 设置图形样式
    plt.style.use('ggplot')
    

    # 绘制每个时间段的组合图
    for idx, period in periods_data.iterrows():
        # 筛选时间段数据
        mask = (ts_data['时间'] >= period['开始时间']) & (ts_data['时间'] <= period['结束时间'])
        period_data = ts_data.loc[mask]
        
        if len(period_data) == 0:
            print(f"时间段 {idx+1} 无数据")
            continue
        
        # 创建双Y轴图表
        fig, ax1 = plt.subplots(figsize=(14, 7))
        
        # 绘制折叠机速度（左轴）
        color = 'tab:blue'
        ax1.set_xlabel('时间')
        ax1.set_ylabel('折叠机实际速度', color=color)
        line1 = ax1.plot(period_data['时间'], period_data['折叠机实际速度'], 
                        color=color, label='折叠机速度')
        ax1.tick_params(axis='y', labelcolor=color)
        
        # 创建右轴并绘制存纸率
        ax2 = ax1.twinx()
        color = 'tab:red'
        ax2.set_ylabel('存纸率', color=color)
        line2 = ax2.plot(period_data['时间'], period_data['存纸率'], 
                        color=color, linestyle='--', label='存纸率')
        ax2.tick_params(axis='y', labelcolor=color)
        
        # 合并图例
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper left')
        
        # 设置标题和格式
        plt.title(f"折叠机速度与存纸率对比\n{period['开始时间']} 至 {period['结束时间']}")
        ax1.xaxis.set_major_formatter(DateFormatter('%H:%M:%S'))
        plt.xticks(rotation=45)
        ax1.grid(True, linestyle='--', alpha=0.7)
        
        # 保存图表
        plt.tight_layout()
        plt.savefig(f"{output_dir_hourly}/时间段_{idx+1}_组合图.png", dpi=800)
        plt.close()
    
    # 绘制全天组合图
    fig, ax1 = plt.subplots(figsize=(18, 8))
    
    # 左轴：折叠机速度
    color = 'tab:blue'
    ax1.set_ylabel('折叠机实际速度', color=color)
    line1 = ax1.plot(ts_data['时间'], ts_data['折叠机实际速度'], 
                    color=color, label='折叠机速度')
    ax1.tick_params(axis='y', labelcolor=color)
    
    # 右轴：存纸率
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('存纸率', color=color)
    line2 = ax2.plot(ts_data['时间'], ts_data['存纸率'], 
                    color=color, linestyle='--', label='存纸率')
    ax2.tick_params(axis='y', labelcolor=color)
    
    # 标记时间段
    for idx, period in periods_data.iterrows():
        ax1.axvspan(period['开始时间'], period['结束时间'], 
                    color='gray', alpha=0.2)
        # 计算时间段中点
        midpoint = period['开始时间'] + (period['结束时间'] - period['开始时间']) / 2
        ax1.text(midpoint, 
                ax1.get_ylim()[1]*0.9,
                f'时段{idx+1}', ha='center', va='center',
                bbox=dict(facecolor='white', alpha=0.8))
    
    # 合并图例
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax1.legend(lines, labels, loc='upper left')
    
    # 设置标题和格式
    date_str = ts_data['时间'].dt.date[0]
    plt.title(f"折叠机速度与存纸率对比 - 全天数据 ({date_str})")
    ax1.xaxis.set_major_formatter(DateFormatter('%H:%M:%S'))
    plt.xticks(rotation=45)
    ax1.grid(True, linestyle='--', alpha=0.7)
    
    # 保存全天图
    plt.tight_layout()
    plt.savefig(f"{output_dir_daily}/全天_组合图.png", dpi=800, bbox_inches='tight')
    plt.close()


#####   事件绘图（初次绘制）

In [65]:
ts_file_path = r'D:\Code_File\Vinda_cunzhijia\存纸架数据汇总.csv'    

periods_file_path1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\折叠机速度周期时间段.csv'    
periods_file_path2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_periods.csv'
target_columns =  ['折叠机实际速度', '存纸率']    

output_dir_hourly1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_hourly'   
output_dir_daily1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_daily'

output_dir_hourly2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_hourly'
output_dir_daily2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_daily'

if not os.path.exists(output_dir_hourly1):    
    os.makedirs(output_dir_hourly1)
if not os.path.exists(output_dir_daily1):    
    os.makedirs(output_dir_daily1)
if not os.path.exists(output_dir_hourly2):    
    os.makedirs(output_dir_hourly2)
if not os.path.exists(output_dir_daily2):    
    os.makedirs(output_dir_daily2)

# 调用函数
plot_time_series_by_periods(ts_file_path, periods_file_path1, target_columns, output_dir_hourly1, output_dir_daily1)
# plot_time_series_by_periods(ts_file_path, periods_file_path2, target_columns, output_dir_hourly2, output_dir_daily2)

# 提取非逼停事件

In [48]:
# 读取文件1
file1 = pd.read_csv(r'D:\Code_File\Vinda_cunzhijia\高低速时间段\折叠机速度周期时间段.csv', parse_dates=['开始时间', '结束时间'])

# 读取文件2
file2 = pd.read_csv(r'D:\Code_File\Vinda_cunzhijia\存纸架数据汇总.csv', parse_dates=['时间'])

# 将时间列转换为datetime对象
file1['开始时间'] = pd.to_datetime(file1['开始时间'])
file1['结束时间'] = pd.to_datetime(file1['结束时间'])
file2['时间'] = pd.to_datetime(file2['时间'])

# 创建两个列表来存储符合和不符合条件的事件
matched_events = []
unmatched_events = []

for index, row in file1.iterrows():
    start_time = row['开始时间']
    
    # 条件1: 开始时存纸率 ≥ 60
    # 获取开始时间点附近的存纸率数据（前后5秒）
    time_window_start = start_time - timedelta(seconds=5)
    time_window_end = start_time + timedelta(seconds=5)
    mask_start = (file2['时间'] >= time_window_start) & (file2['时间'] <= time_window_end)
    data_start = file2[mask_start]
    
    condition1 = False
    if not data_start.empty:
        # 检查存纸率是否 ≥ 60
        avg_paper_rate_start = data_start['存纸率'].mean()
        condition1 = avg_paper_rate_start >= 60
    
    # 条件2: 开始时间后的2分钟内存纸率有下降趋势
    time_after = start_time + timedelta(minutes=2)
    mask_after = (file2['时间'] >= start_time) & (file2['时间'] <= time_after)
    data_after = file2[mask_after]
    
    condition2 = False
    if len(data_after) > 1:
        # 计算线性回归斜率判断趋势
        x = (data_after['时间'] - start_time).dt.total_seconds().values.reshape(-1, 1)
        y = data_after['存纸率'].values
        
        # 简单斜率计算（首点和末点）
        start_rate = y[0]
        end_rate = y[-1]
        slope = (end_rate - start_rate) / (x[-1][0] - x[0][0])
        
        condition2 = slope < 0  # 斜率为负表示下降趋势
    
    # 如果两个条件都满足，添加到匹配事件列表
    if condition1 and condition2:
        matched_events.append(row)
    else:
        unmatched_events.append(row)

# 将结果转换为DataFrame
matched_df = pd.DataFrame(matched_events)
unmatched_df = pd.DataFrame(unmatched_events)

# 保存结果到新的CSV文件
matched_df.to_csv('D:\Code_File\Vinda_cunzhijia\高低速时间段\逼停事件.csv', index=False)
unmatched_df.to_csv('D:\Code_File\Vinda_cunzhijia\高低速时间段\其他事件.csv', index=False)

print(f"找到{len(matched_events)}个逼停事件，{len(unmatched_events)}个其他事件")

找到16个逼停事件，104个其他事件


# 事件绘图（逼停事件）

In [66]:
ts_file_path = r'D:\Code_File\Vinda_cunzhijia\存纸架数据汇总.csv'    

periods_file_path1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\逼停事件.csv' 
periods_file_path2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_periods.csv'
target_columns =  ['折叠机实际速度', '存纸率']    

output_dir_hourly1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_hourly_v1'   
output_dir_daily1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_daily_v1'

output_dir_hourly2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_hourly'
output_dir_daily2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_daily'

if not os.path.exists(output_dir_hourly1):    
    os.makedirs(output_dir_hourly1)
if not os.path.exists(output_dir_daily1):    
    os.makedirs(output_dir_daily1)
if not os.path.exists(output_dir_hourly2):    
    os.makedirs(output_dir_hourly2)
if not os.path.exists(output_dir_daily2):    
    os.makedirs(output_dir_daily2)

# 调用函数
plot_time_series_by_periods(ts_file_path, periods_file_path1, target_columns, output_dir_hourly1, output_dir_daily1)
# plot_time_series_by_periods(ts_file_path, periods_file_path2, target_columns, output_dir_hourly2, output_dir_daily2)

### 逼停事件网页版（折叠机、存纸率、小包机速度对比）

In [67]:

def plot_time_series_by_periods_interactive(
    ts_file_path,          # 时间序列文件路径
    periods_file_path,     # 时间段文件路径
    target_columns,        # 要绘制的列（列表形式）
    output_dir_hourly,     # 输出HTML文件的目录（小时级）
    output_dir_daily       # 输出HTML文件的目录（天级）
):
    """   
    参数:
        ts_file_path: 时间序列文件路径
        periods_file_path: 时间段文件路径
        target_columns: 要绘制的列名列表
        output_dir_hourly: 输出小时级HTML文件的目录
        output_dir_daily: 输出天级HTML文件的目录
    """
    
    # 读取数据
    ts_data = pd.read_csv(ts_file_path, parse_dates=['时间'])
    periods_data = pd.read_csv(periods_file_path, parse_dates=['开始时间', '结束时间'])
    
    # 创建包含所有小时段图表的HTML内容
    html_content = """
    <!DOCTYPE html>
    <html>
    <head>
        <title>设备速度与存纸率对比 - 所有时间段</title>
        <style>
            .chart-container {
                margin-bottom: 50px;
                border: 1px solid #ddd;
                padding: 15px;
                border-radius: 5px;
            }
            h2 {
                color: #333;
                margin-top: 30px;
            }
        </style>
    </head>
    <body>
        <h1 style="text-align: center;">设备速度与存纸率对比 - 所有时间段</h1>
    """
    
    # 绘制每个时间段的组合图
    for idx, period in periods_data.iterrows():
        # 筛选时间段数据
        mask = (ts_data['时间'] >= period['开始时间']) & (ts_data['时间'] <= period['结束时间'])
        period_data = ts_data.loc[mask]
        
        if len(period_data) == 0:
            print(f"时间段 {idx+1} 无数据")
            continue
        
        # 创建带有双Y轴的图表
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        # 添加折叠机速度轨迹（左轴）
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['折叠机实际速度'],
                name='折叠机速度',
                line=dict(color='blue'),
                mode='lines'
            ),
            secondary_y=False,
        )
        
        # 添加四个小包机速度轨迹（左轴）
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['1#小包机实际速度'],
                name='1#小包机速度',
                line=dict(color='green'),
                mode='lines'
            ),
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['2#小包机实际速度'],
                name='2#小包机速度',
                line=dict(color='orange'),
                mode='lines'
            ),
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['3#小包机主机实际速度'],
                name='3#小包机速度',
                line=dict(color='purple'),
                mode='lines'
            ),
            secondary_y=False,
        )
        
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['4#小包机主机实际速度'],
                name='4#小包机速度',
                line=dict(color='brown'),
                mode='lines'
            ),
            secondary_y=False,
        )
        
        # 添加存纸率轨迹（右轴）
        fig.add_trace(
            go.Scatter(
                x=period_data['时间'],
                y=period_data['存纸率'],
                name='存纸率',
                line=dict(color='red', dash='dot'),
                mode='lines'
            ),
            secondary_y=True,
        )
        
        # 设置轴标签
        fig.update_yaxes(title_text="设备速度", secondary_y=False)
        fig.update_yaxes(title_text="存纸率", secondary_y=True)
        
        # 设置标题和布局
        fig.update_layout(
            title_text=f"时间段 {idx+1}: {period['开始时间']} 至 {period['结束时间']}",
            hovermode="x unified",
            template="plotly_white",
            height=500,
            width=900,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )
        
        # 将图表添加到HTML内容中
        html_content += f"""
        <div class="chart-container">
            <h2>时间段 {idx+1}: {period['开始时间']} 至 {period['结束时间']}</h2>
            {fig.to_html(full_html=False, include_plotlyjs='cdn')}
        </div>
        """
    
    
    
    # 完成HTML内容
    html_content += """
    </body>
    </html>
    """
    
    # 保存所有图表到一个HTML文件
    combined_html_path = os.path.join(output_dir_hourly, "所有时间段_组合图.html")
    with open(combined_html_path, "w", encoding="utf-8") as f:
        f.write(html_content)
    
    # 自动在浏览器中打开生成的HTML文件
    webbrowser.open_new_tab('file://' + os.path.abspath(combined_html_path))
    
    # 单独保存全天图
    daily_html_path = os.path.join(output_dir_daily, "全天_组合图.html")
    fig.write_html(daily_html_path)

# 文件路径配置
ts_file_path = r'D:\Code_File\Vinda_cunzhijia\存纸架数据汇总.csv'    
periods_file_path1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\逼停事件.csv'    
periods_file_path2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_periods.csv'
target_columns =  ['折叠机实际速度', '存纸率', '1#小包机实际速度', '2#小包机实际速度', '3#小包机主机实际速度', '4#小包机主机实际速度']    

output_dir_hourly1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_hourly'   
output_dir_daily1 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\low_speed_daily'
output_dir_hourly2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_hourly'
output_dir_daily2 = r'D:\Code_File\Vinda_cunzhijia\高低速时间段\high_speed_daily'

# 创建输出目录
if not os.path.exists(output_dir_hourly1):    
    os.makedirs(output_dir_hourly1)
if not os.path.exists(output_dir_daily1):    
    os.makedirs(output_dir_daily1)
if not os.path.exists(output_dir_hourly2):    
    os.makedirs(output_dir_hourly2)
if not os.path.exists(output_dir_daily2):    
    os.makedirs(output_dir_daily2)

# 调用函数
plot_time_series_by_periods_interactive(ts_file_path, periods_file_path1, target_columns, output_dir_hourly1, output_dir_daily1)
# plot_time_series_by_periods_interactive(ts_file_path, periods_file_path2, target_columns, output_dir_hourly2, output_dir_daily2)